Displaying a simple GIS map using Folium


In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

# Load the shapefile
gdf = gpd.read_file('../shapefiles/cb_2018_us_county_500k.shp')

# Load the population data
population_data = pd.read_csv('../data/Population_Change_County.csv')

# Merge the GeoDataFrame with the population data DataFrame on the common identifier
merged_gdf = gdf.merge(population_data, left_on='AFFGEOID', right_on='Geography')
merged_gdf

# Convert the merged GeoDataFrame to GeoJSON
geojson_data = merged_gdf.to_json()

# Create a base map
initial_coordinates = [37.0902, -95.7129]  # Coordinates of the geographic center of the US
mymap = folium.Map(location=initial_coordinates, zoom_start=4)

def create_choropleth(geojson_data, data, column, name, color):
    layer = folium.Choropleth(
        geo_data=geojson_data,
        data=data,
        columns=['AFFGEOID', column],
        key_on='feature.properties.AFFGEOID',
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=name + ' Population'
    )
    # Add tooltip
    tooltip = GeoJsonTooltip(
        fields=['NAME', column],
        aliases=['County:', 'Population:'],
        localize=True
    )
    layer.geojson.add_child(tooltip)
    return layer

# Add choropleth layers for the year 2000 and 2020
layer_2000 = create_choropleth(geojson_data, merged_gdf, 'Sum of Population2000', '2000', 'BuGn')
layer_2020 = create_choropleth(geojson_data, merged_gdf, 'Sum of Population2020', '2020', 'OrRd')

# Add layers to the map
layer_2000.add_to(mymap)
layer_2020.add_to(mymap)

# Add a layer control panel to the map
folium.LayerControl().add_to(mymap)

# Save the map to an HTML file
html_save_path = '../output/both_years_map.html'
mymap.save(html_save_path)

# Display the map
mymap


In [ ]:
import folium
import branca.colormap as cm
import pandas as pd
import geopandas as gpd
from folium.features import GeoJsonTooltip

# Load the shapefile and population data, and merge them
gdf = gpd.read_file('../shapefiles/cb_2018_us_county_500k.shp')
population_data = pd.read_csv('../data/Population_Change_County.csv')
merged_gdf = gdf.merge(population_data, left_on='AFFGEOID', right_on='Geography')

# Convert the merged GeoDataFrame to GeoJSON
geojson_data = merged_gdf.to_json()

# Create a base map
initial_coordinates = [37.0902, -95.7129]  # Coordinates of the geographic center of the US
map = folium.Map(location=initial_coordinates, zoom_start=4)

# Define a symmetric colormap
max_abs_val = max(abs(merged_gdf['Sum of Percentage Difference'].min()), abs(merged_gdf['Sum of Percentage Difference'].max()))
colormap = cm.LinearColormap(colors=['red', 'white', 'blue'], vmin=-max_abs_val, vmax=max_abs_val)

# Custom style function
def style_function(feature):
    value = feature['properties']['Sum of Percentage Difference']
    return {
        'fillColor': colormap(value),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# Add tooltip
tooltip = GeoJsonTooltip(
    fields=['NAME', 'Sum of Percentage Difference'],
    aliases=['County:', 'Percentage Difference:'],
    localize=True
)

# Add the GeoJson layer to the map
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=tooltip
).add_to(map)

# Add the colormap to the map
colormap.caption = 'Percentage Difference'
colormap.add_to(map)

# Add a layer control panel to the map
folium.LayerControl().add_to(map)

# Save the map to an HTML file
html_save_path = '../output/difference_map.html'
map.save(html_save_path)

map


In [ ]:
#same plot using plotly
import geopandas as gpd
import pandas as pd
import plotly.express as px
import json

# Load the shapefile and population data, and merge them
gdf = gpd.read_file('../shapefiles/cb_2018_us_county_500k.shp')
population_data = pd.read_csv('../data/Population_Change_County.csv')
merged_gdf = gdf.merge(population_data, left_on='AFFGEOID', right_on='Geography')

# Make sure the GeoDataFrame is in latitude-longitude
if merged_gdf.crs != 'epsg:4326':
    merged_gdf = merged_gdf.to_crs(epsg=4326)
# Prepare the GeoJSON
geojson = json.loads(merged_gdf.to_json())

# Create the figure
fig = px.choropleth_mapbox(
    merged_gdf,
    geojson=geojson,
    locations=merged_gdf.index,  # Use the DataFrame index for locations
    color='Sum of Population2000',  # Initial color for the year 2000
    color_continuous_scale='Blues',
    mapbox_style='carto-positron',
    zoom=3,
    center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.5
)

# Add a second layer for the year 2020
fig.add_choroplethmapbox(
    geojson=geojson,
    locations=merged_gdf.index,
    z=merged_gdf['Sum of Population2020'],
    colorscale='Reds',
    zmin=merged_gdf['Sum of Population2000'].min(),
    zmax=merged_gdf['Sum of Population2000'].max(),
    visible=False  # Start with this layer hidden
)

# Create the slider
steps = []
for i, year in enumerate(['Sum of Population2000', 'Sum of Population2020']):
    step = dict(
        method='update',
        args=[{'visible': [False, False]},
              {'title': f'Population for the year: {year[-4:]}'}
             ],
        label=str(year[-4:])
    )
    step['args'][0]['visible'][i] = True  # Toggle i'th trace to 'visible'
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    steps=steps
)]

fig.update_layout(sliders=sliders)

# Hide the axis ticks and labels
fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})

# Show the figure
fig.show()
